# Economías del crimen Bogotá

Establecer tamaño de la muestra y grupos
Variables de resutlado

In [2]:
devtools::install_github('vikjam/pwrcalc')
install.packages("pwr")
require("pwr")
require("pwrcalc")


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpOu2OLS/remotesf1167aaf43/vikjam-pwrcalc-031b6ef/DESCRIPTION’ ... OK
* preparing ‘pwrcalc’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘pwrcalc_0.0.0.9001.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: pwr

Loading required package: pwrcalc

Loading required package: magrittr



In [3]:

power = 0.8                                                                     #SPECIFY - desired power
nratio = 2                                                                     #SPECIFY - the ratio of the size of the treatment group to control group
alpha =0.05                                                                     #SPECIFY - significance level
p = nratio/(1+nratio)

baseline_mean <-  0                              #Record the mean of the outcome variable at baseline
baseline_sd <- 1                                      #Record the standard deviation of the outcome variable at baseline

expected_effect = 0.05*baseline_sd                                             #The expected effect should be specified based on the intervention and the cost.
                                                                                #Here it is 0.3 times the sd
treated_mean <- expected_effect + baseline_mean

base_model = twomeans(m1 = baseline_mean, m2 = treated_mean, sd = baseline_sd,
                        nratio=nratio, power=power, sig.level = alpha)

base_model


     Two-sample t-test power calculation 

             m1 = 0
             m2 = 0.05
             n1 = 4710
             n2 = 9419
      sig.level = 0.05
          power = 0.8
    alternative = two.sided

NOTE: 
m1 and m2 are the means of group 1 and 2, respectively.
n1 and n2 are the obs. of group 1 and 2, respectively.


# Construyendo el dataset simulado

In [4]:
devtools::install_github('DeclareDesign/DeclareDesign')
require("DeclareDesign")

Rcpp      (NA -> 1.0.11   ) [CRAN]
RcppEigen (NA -> 0.3.3.9.4) [CRAN]
Formula   (NA -> 1.2-5    ) [CRAN]
estimatr  (NA -> 1.0.0    ) [CRAN]
fabricatr (NA -> 1.0.0    ) [CRAN]
randomizr (NA -> 1.0.0    ) [CRAN]


Installing 6 packages: Rcpp, RcppEigen, Formula, estimatr, fabricatr, randomizr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpOu2OLS/remotesf119dbea66/DeclareDesign-DeclareDesign-9e99530/DESCRIPTION’ ... OK
* preparing ‘DeclareDesign’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘DeclareDesign_1.0.4.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DeclareDesign

Loading required package: randomizr

Loading required package: fabricatr

Loading required package: estimatr



In [5]:
devtools::install_github('DeclareDesign/DesignLibrary')
require("DesignLibrary")


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpOu2OLS/remotesf169d1be96/DeclareDesign-DesignLibrary-b3676fb/DESCRIPTION’ ... OK
* preparing ‘DesignLibrary’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘DesignLibrary_0.1.10.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DesignLibrary



In [6]:
design_two_arm<-DesignLibrary::two_arm_designer()


In [7]:
design_two_arm


Research design declaration summary

Step 1 (population): declare_population(N = N, u_0 = rnorm(N), u_1 = rnorm(n = N, mean = rho * u_0, sd = sqrt(1 - rho^2))) 

Step 2 (potential outcomes): declare_potential_outcomes(Y ~ (1 - Z) * (u_0 * control_sd + control_mean) + Z * (u_1 * treatment_sd + treatment_mean)) 

Step 3 (inquiry): declare_inquiry(ATE = mean(Y_Z_1 - Y_Z_0)) -------------------

Step 4 (assignment): declare_assignment(Z = complete_ra(N, prob = assignment_prob)) 

Step 5 (reveal): declare_reveal() ----------------------------------------------

Step 6 (estimator): declare_estimator(Y ~ Z, inquiry = estimand) ---------------

Run of the design:

 inquiry estimand estimator term estimate std.error statistic  p.value conf.low
     ATE        1 estimator    Z    0.893     0.204      4.38 3.03e-05    0.488
 conf.high df outcome
       1.3 98       Y


In [8]:
data_mundo1<-draw_data(design_two_arm)
head(data_mundo1)


,ID,u_0,u_1,Y_Z_0,Y_Z_1,Z,Y
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,001,-0.6026096,-0.6026096,-0.6026096,0.3973904,0,-0.6026096
2,002,0.7390408,0.7390408,0.7390408,1.7390408,1,1.7390408
3,003,-0.4258019,-0.4258019,-0.4258019,0.5741981,0,-0.4258019
4,004,0.3070082,0.3070082,0.3070082,1.3070082,1,1.3070082
5,005,-1.6893876,-1.6893876,-1.6893876,-0.6893876,0,-1.6893876
6,006,-0.2635513,-0.2635513,-0.2635513,0.7364487,1,0.7364487


In [9]:
lm(Y~Z,data=data_mundo1)



Call:
lm(formula = Y ~ Z, data = data_mundo1)

Coefficients:
(Intercept)            Z  
   0.007375     1.064855  


In [10]:
panels <- fabricate(
  listings = add_level(N = 17000, listing_fe = runif(N, -.05, .05)),
  hours = add_level(N = 3, hour_shock = runif(N, -0.0, 0.0), nest = FALSE),
  obs = cross_levels(
    by = join_using(listings, hours),
    U =rnorm(N, 0, .02),
    epsilon = listing_fe + hour_shock + U
  )

)

require(tidyverse)
panels  %>% arrange(listings)

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract()   masks magrittr::extract()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ purrr::set_names() masks magrittr::set_names()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


listings,listing_fe,hours,hour_shock,obs,U,epsilon
<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
00001,-0.04205437,1,0,00001,0.0060475292,-0.036006841
00001,-0.04205437,2,0,17001,0.0074893332,-0.034565037
00001,-0.04205437,3,0,34001,0.0236509660,-0.018403404
00002,-0.01180135,1,0,00002,-0.0192771573,-0.031078508
00002,-0.01180135,2,0,17002,0.0152182624,0.003416912
00002,-0.01180135,3,0,34002,0.0140059566,0.002204606
00003,-0.01585378,1,0,00003,-0.0248751707,-0.040728947
00003,-0.01585378,2,0,17003,-0.0013271359,-0.017180912
00003,-0.01585378,3,0,34003,0.0033959328,-0.012457843


In [35]:

design0<-declare_model(panels,
                       potential_outcomes(Y ~ rbinom(n = N, size = 1, prob = 0.6-0.05* Z+epsilon)))+
                      declare_inquiry(ATE=mean(Y_Z_1-Y_Z_0))  + declare_assignment(Z=block_ra(blocks=listings)) +
                      declare_measurement(Y=reveal_outcomes(Y~Z))+
                      declare_estimator(Y~Z,term="Z", .method=lm, label="OLS") +
                        declare_estimator(Y~Z+factor(listings)+factor(days),term="Z", .method=lm, label="FE") +
                        declare_estimator(Y~Z,term="Z", .method=glm, family="binomial", label="Logit")
diagnose_design(design0,sims=30)


Research design diagnosis based on 30 simulations. Diagnosis completed in 6 secs. Diagnosand estimates with bootstrapped standard errors in parentheses (100 replicates).

  Design Inquiry Estimator Term N Sims Mean Estimand Mean Estimate   Bias
 design0     ATE        FE    Z     30         -0.05         -0.05   0.00
                                              (0.00)        (0.01) (0.00)
 design0     ATE     Logit    Z     30         -0.05         -0.19  -0.14
                                              (0.00)        (0.02) (0.02)
 design0     ATE       OLS    Z     30         -0.05         -0.05   0.00
                                              (0.00)        (0.01) (0.00)
 SD Estimate   RMSE  Power Coverage
        0.03   0.02   0.30     0.97
      (0.00) (0.00) (0.07)   (0.03)
        0.12   0.17   0.33     0.87
      (0.01) (0.01) (0.08)   (0.06)
        0.03   0.02   0.33     0.97
      (0.00) (0.00) (0.08)   (0.03)

In [9]:
library(fabricatr)
library(tidyverse)

# Parámetros del análisis de potencia
m1 = 0
m2 = 0.13
n1 = 697
n2 = 1394
sig.level = 0.05
power = 0.8
alternative = "two.sided"
N_total = n1 + n2
# Creación de datos simulados

design<- declare_model(
            N=N_total,
            U=rnorm(N,sd=1),
            potential_outcomes(Y~ 0.03*Z +U )) +
            declare_inquiry(ATE=Y_Z_1-Y_Z_0) +
            declare_assignment(Z=complete_ra(N,prob=0.5)) +
            declare_measurement(Y=reveal_outcomes(Y~Z))

head(draw_data(design),10)

,ID,U,Y_Z_0,Y_Z_1,Z,Y
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,0001,-0.28955077,-0.28955077,-0.25955077,1,-0.25955077
2,0002,-1.45267289,-1.45267289,-1.42267289,1,-1.42267289
3,0003,-1.99922927,-1.99922927,-1.96922927,1,-1.96922927
4,0004,-0.07584835,-0.07584835,-0.04584835,0,-0.07584835
5,0005,1.17805829,1.17805829,1.20805829,1,1.20805829
6,0006,0.95570407,0.95570407,0.98570407,1,0.98570407
7,0007,0.40132083,0.40132083,0.43132083,0,0.40132083
8,0008,1.61685470,1.61685470,1.64685470,0,1.61685470
9,0009,-0.86004856,-0.86004856,-0.83004856,1,-0.83004856


In [11]:

panels <- fabricate(
  listings = add_level(N = N_total, listing_fe = runif(N, -.1, .1)),
  years = add_level(N = 2, year_shock = runif(N, -.05, .05), nest = FALSE),
  obs = cross_levels(
    by = join_using(listings, years),
    U =rnorm(N, 0, .1),
    epsilon = listing_fe + year_shock + U
  )


)

require(tidyverse)
panels  %>% arrange(listings)

listings,listing_fe,years,year_shock,obs,U,epsilon
<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
0001,-0.0871545432,1,0.017380010,0001,0.109831617,0.040057084
0001,-0.0871545432,2,0.009651702,2092,0.001163267,-0.076339574
0002,0.0392043280,1,0.017380010,0002,-0.069770335,-0.013185997
0002,0.0392043280,2,0.009651702,2093,0.088788778,0.137644808
0003,-0.0364938927,1,0.017380010,0003,-0.115644193,-0.134758076
0003,-0.0364938927,2,0.009651702,2094,0.093391333,0.066549143
0004,-0.0560501603,1,0.017380010,0004,-0.090616799,-0.129286949
0004,-0.0560501603,2,0.009651702,2095,-0.144511435,-0.190909893
0005,-0.0513330272,1,0.017380010,0005,0.020147554,-0.013805463


In [36]:
panels <- fabricate(
  listings = add_level(
    N = 17000,
    listing_fe = runif(N, -0.05, 0.05)
  ),
  year = add_level(N = 2, year = c(0, 1), year_shock = runif(N, -0.0, 0.0), nest = FALSE),
  obs = cross_levels(
    by = join_using(listings, year),
    U = rnorm(N, 0, .02),
    epsilon = listing_fe + year_shock + U
  )
)


In [46]:
# Ahora, ajusta el modelo potencial de resultados para incorporar nativo e inmigrante
design0 <- declare_model(
  panels,
  potential_outcomes(Y~ 0.13*Z +U )) +
  declare_inquiry(
          ATE = mean(Y_Z_1-Y_Z_0)
          ) +
        declare_sampling(S = complete_rs(N = N, n = 500)) +
        declare_assignment(Z = block_ra(blocks = listings)) +
        declare_measurement(Y = reveal_outcomes(Y ~ Z)) +
        declare_estimator(model = lm, formula = Y ~ Z, term = "Z", label = "OLS") +
        declare_estimator(model = lm, formula = Y ~ Z + factor(listings) + factor(hours), term = "Z", label = "FE")


In [47]:
design0<-declare_model(panels,
                       potential_outcomes(Y ~ rbinom(n = N, size = 1, prob = 0.6-0.05* Z+epsilon)))+ NULL

head(draw_data(design0) %>% arrange(listings),20)

,listings,listing_fe,year,year_shock,obs,U,epsilon,Y_Z_0,Y_Z_1
,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<int>,<int>
1,00001,0.0007756331,0,0,00001,-2.180038e-02,-0.021024749,1,1
2,00001,0.0007756331,1,0,17001,8.089068e-03,0.008864701,1,1
3,00002,0.0207074376,0,0,00002,-7.900356e-05,0.020628434,1,0
4,00002,0.0207074376,1,0,17002,-3.718619e-02,-0.016478751,0,1
5,00003,-0.0358737486,0,0,00003,-5.592365e-03,-0.041466114,0,1
6,00003,-0.0358737486,1,0,17003,1.416093e-02,-0.021712819,1,1
7,00004,0.0443822315,0,0,00004,-3.119850e-02,0.013183730,1,1
8,00004,0.0443822315,1,0,17004,-2.042906e-02,0.023953176,1,0
9,00005,0.0346535246,0,0,00005,-1.896665e-02,0.015686877,0,0


In [49]:
install.packages("rdss")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘dataverse’, ‘prediction’




In [52]:
install.packages("DIDmultiplegt")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘DIDmultiplegt’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


# Solamente el Año de la intervencion comparaicón con controles puros de distancia

In [ ]:
library(rdss) # for helper functions

N_time_periods <- 2
N_units <- 2000 # Total de unidades
N_treated_units <- 1400 # Unidades tratadas
N_untreated_units <- 600 # Unidades no tratadas
N_time_periods <- 2 # Dos periodos

# Crear un vector con las unidades tratadas (1) y no tratadas (0)
treatment_status <- c(rep(1, N_treated_units), rep(0, N_untreated_units))

set.seed(123) # Para reproducibilidad
D_unit <- sample(treatment_status, N_units, replace = FALSE)
# Crear un vector con hotspots grandes e influyentes (1) y pequeños y poco influyentes (0)
hotspot_type <- c(rep(1, N_units / 2), rep(0, N_units / 2))
set.seed(124) # Usar una semilla diferente para asegurar variabilidad
hotspot_type <- sample(hotspot_type, N_units, replace = FALSE)



declaration_16.3 <-
  declare_model(
    units = add_level(
      N = N_units,
      U_unit = rnorm(N),
      D_unit = sample(treatment_status, N_units, replace = FALSE),
      U_unit = rnorm(N_units)  # Ruido aleatorio para cada unidad
    ),
    periods = add_level(
      N = N_time_periods,
      U = rnorm(N),
      nest = FALSE
    ),
    unit_period = cross_levels(
      by = join_using(units, periods),
      D = if_else(periods == 2 & D_unit == 1, 1, 0),  # Asignación del tratamiento en el segundo periodo
      potential_outcomes(
        Y ~ 0.87 * D + U_unit,
        conditions = list(D = c(0, 1))
      )  # Efecto del tratamiento y ruido
    )
  ) +
  declare_inquiry(
    ATT = mean(Y_D_1 - Y_D_0),
    subset = D == 1
  ) +
  declare_measurement(Y = reveal_outcomes(Y ~ D)) +
  declare_estimator(
    Y ~ D, fixed_effects = ~ units + periods,
    .method = lm_robust,
    inquiry = c("ATT"),
    label = "twoway-fe"
  )

## El Año luedo de la itenrevencion Diferencais en Diferencias

In [ ]:
library(DeclareDesign)

# Definir el número total de unidades y distancias
N_units <- 2000
distances <- c(0, 50, 250, 500)
N_distances <- length(distances)
N_total_observations <- N_units * N_distances

# Expandir las unidades para cada distancia
expanded_units <- rep(1:N_units, times = N_distances)
expanded_distance <- rep(distances, each = N_units)

# Simulando el ruido aleatorio y los efectos de distancia
set.seed(123)
U_unit = rnorm(N_total_observations)  # Ruido aleatorio para cada observación
effect_by_distance <- c(-0.1, -0.05, 0.05, 0.1) # Efecto diferenciado por distancia

# Declaración del modelo
declaration <-
  declare_model(
    units = add_level(
      N = N_total_observations,
      Unit_ID = expanded_units,
      Distance = expanded_distance,
      Y_potential = U_unit + ifelse(Distance == 0, effect_by_distance[1],
                                    ifelse(Distance == 50, effect_by_distance[2],
                                           ifelse(Distance == 250, effect_by_distance[3],
                                                  effect_by_distance[4])))
    )
  ) +
  declare_measurement(Y = Y_potential) +
  declare_estimator(
    Y ~ Distance,
    model = lm_robust,
    inquiry = "mean_effect",
    label = "linear_model"
  ) +
  declare_estimator(
    Y ~ Distance,
    model = lm_robust,
    fixed_effects = ~ Unit_ID,
    inquiry = "fixed_effects",
    label = "fixed_effects_model"
  ) +
  declare_estimator(
    Y ~ Distance + I(Distance^2),
    model = lm_robust,
    inquiry = "non_linear_effect",
    label = "non_linear_model"
  )

# Ejecutar el diseño y analizar los resultados
diagnosis <- diagnose_design(declaration)


Loading required package: randomizr

Loading required package: fabricatr

Loading required package: estimatr

Warning message in fn(data, ~(Y ~ Distance), model = ~lm_robust):
“The argument 'model = ' is deprecated. Please use '.method = ' instead.”
Warning message in fn(data, ~(Y ~ Distance), fixed_effects = ~~Unit_ID, model = ~lm_robust):
“The argument 'model = ' is deprecated. Please use '.method = ' instead.”
Warning message in fn(data, ~(Y ~ Distance + I(Distance^2)), model = ~lm_robust):
“The argument 'model = ' is deprecated. Please use '.method = ' instead.”
Warning message in fn(data, ~(Y ~ Distance), model = ~lm_robust):
“The argument 'model = ' is deprecated. Please use '.method = ' instead.”
Warning message in fn(data, ~(Y ~ Distance), fixed_effects = ~~Unit_ID, model = ~lm_robust):
“The argument 'model = ' is deprecated. Please use '.method = ' instead.”
Warning message in fn(data, ~(Y ~ Distance + I(Distance^2)), model = ~lm_robust):
“The argument 'model = ' is deprecated

In [ ]:
head(draw_data(declaration_16.3) %>% arrange(units),20)

In [100]:

multiple_desing<-redesign(declaration_16.3,ates=c(0.5,1,1.5),
sample_size=c(20,40,50))

diagnosis <- diagnose_design(multiple_desing,sims=2)
tidy(diagnosis)

Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”
Warning message:
“We recommend you choose a number of simulations higher than 30.”


design,inquiry,estimator,outcome,term,ates,sample_size,diagnosand,estimate,std.error,conf.low,conf.high
<fct>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
design_1,ATT,twoway-fe,Y,D,0.5,20,mean_estimand,8.700000e-01,0.000000e+00,8.700000e-01,8.700000e-01
design_1,ATT,twoway-fe,Y,D,0.5,20,mean_estimate,8.700000e-01,1.097318e-14,8.700000e-01,8.700000e-01
design_1,ATT,twoway-fe,Y,D,0.5,20,bias,2.681189e-14,1.097918e-14,9.658940e-15,4.396483e-14
design_1,ATT,twoway-fe,Y,D,0.5,20,sd_estimate,2.425806e-14,1.203309e-14,0.000000e+00,2.425806e-14
design_1,ATT,twoway-fe,Y,D,0.5,20,rmse,3.182924e-14,1.178294e-14,9.658940e-15,4.396483e-14
design_1,ATT,twoway-fe,Y,D,0.5,20,power,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
design_1,ATT,twoway-fe,Y,D,0.5,20,coverage,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
design_2,ATT,twoway-fe,Y,D,1.0,20,mean_estimand,8.700000e-01,0.000000e+00,8.700000e-01,8.700000e-01
design_2,ATT,twoway-fe,Y,D,1.0,20,mean_estimate,8.700000e-01,4.275192e-16,8.700000e-01,8.700000e-01


## Efectos

In [68]:
library(DeclareDesign)

# Definir el número total de unidades y periodos
N_units <- 2000
N_treated_units <- 1400
N_untreated_units <- 600
N_time_periods <- 2

# Crear un vector con las unidades tratadas (1) y no tratadas (0)
treatment_status <- c(rep(1, N_treated_units), rep(0, N_untreated_units))

# Asignar aleatoriamente el estado de tratamiento a las unidades
set.seed(123)
D_unit <- sample(treatment_status, N_units, replace = FALSE)

# Declarar el modelo
declaration_16.3 <-
  declare_model(
    units = add_level(
      N = N_units,
      U_unit = rnorm(N_units),
      D_unit = D_unit  # Estado de tratamiento asignado previamente
    ),
    periods = add_level(
      N = N_time_periods,
      nest = FALSE
    ),
    unit_period = cross_levels(
      by = join_using(units, periods),
      D = if_else(periods == 2 & D_unit == 1, 1, 0)  # Tratamiento en el segundo periodo para unidades seleccionadas
    )
  ) +
  # Agrega aquí las declaraciones de investigación, medición y estimadores

# Ejecutar el diseño
diagnosis <- diagnose_design(declaration_16.3)


ERROR: ignored

In [ ]:
library(DeclareDesign)

# Definir el número total de unidades y periodos
N_units <- 2000
N_treated_units <- 1400
N_untreated_units <- 600
N_time_periods <- 2

# Crear un vector con las unidades tratadas (1) y no tratadas (0)
treatment_status <- c(rep(1, N_treated_units), rep(0, N_untreated_units))

# Asignar aleatoriamente el estado de tratamiento a las unidades
set.seed(123)
D_unit <- sample(treatment_status, N_units, replace = FALSE)

# Declarar el modelo
declaration_16.3 <-
  declare_model(
    units = add_level(
      N = N_units,
      U_unit = rnorm(N_units),
      D_unit = D_unit  # Estado de tratamiento asignado previamente
    ),
    distancia = add_level(
      N = c(0,250,500),
      nest = FALSE
    ),
    unit_period = cross_levels(
      by = join_using(units, distancia),
    )
  ) +
  # Agrega aquí las declaraciones de investigación, medición y estimadores

# Ejecutar el diseño
diagnosis <- diagnose_design(declaration_16.3)


In [15]:

N_units <- 2000 # Número original de unidades
distances <- c(0, 50, 250, 500) # Distancias al hotspot
N_distances <- length(distances)
N_time_periods <- 2 # Dos periodos
N_total_observations <- N_units * N_distances * N_time_periods

# Crear un vector con las unidades tratadas (1) y no tratadas (0)
treatment_status <- c(rep(1, N_units * 70 / 100), rep(0, N_units * 30 / 100))
set.seed(123)
D_unit <- sample(treatment_status, N_units, replace = FALSE)

# Expandir las unidades para cada distancia y cada periodo
expanded_units <- rep(1:N_units, times = N_distances * N_time_periods)
expanded_D_unit <- rep(D_unit, each = N_distances * N_time_periods)
expanded_distance <- rep(distances, each = N_units * N_time_periods)

# Declaración del modelo
declaration_16.3 <-
  declare_model(
    units = add_level(
      N = N_total_observations,
      D_unit = expanded_D_unit,
      U_unit = rnorm(N_total_observations)  # Ruido aleatorio para cada observación
    ),
    distancia = add_level(
      N = 4,
      U = rnorm(N_time_periods),
      nest = FALSE

    ),
    unit_period = cross_levels(
      by = join_using(units, distancia),
      D = if_else(periods == 2 & D_unit == 1, 1, 0),  # Asignación del tratamiento en el segundo periodo
      potential_outcomes(
        Y ~ 0.87 * D + 0.01 * Distance + U_unit,
        conditions = list(D = c(0, 1))
      )  # Efecto del tratamiento, distancia y ruido
    )
  ) +
  declare_inquiry(
    ATT = mean(Y_D_1 - Y_D_0),
    subset = D == 1
  ) +
  declare_measurement(Y = reveal_outcomes(Y ~ D)) +
  declare_estimator(
    Y ~ D + Distance, fixed_effects = ~ units + periods,
    .method = lm_robust,
    inquiry = c("ATT"),
    label = "twoway-fe"
  )



In [16]:
head(draw_data(declaration_16.3) %>% arrange(units),20)

,units,Unit_ID,D_unit,Distance,U_unit,periods,U,unit_period,D,Y_D_0,Y_D_1,Y
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,00001,1,1,0,-0.3184583,1,2.4524646,00001,0,-0.3184583,0.5515417,-0.3184583
2,00001,1,1,0,-0.3184583,2,0.8828973,16001,1,-0.3184583,0.5515417,0.5515417
3,00002,2,1,0,-0.2205419,1,2.4524646,00002,0,-0.2205419,0.6494581,-0.2205419
4,00002,2,1,0,-0.2205419,2,0.8828973,16002,1,-0.2205419,0.6494581,0.6494581
5,00003,3,1,0,-0.5796481,1,2.4524646,00003,0,-0.5796481,0.2903519,-0.5796481
6,00003,3,1,0,-0.5796481,2,0.8828973,16003,1,-0.5796481,0.2903519,0.2903519
7,00004,4,1,0,0.1889180,1,2.4524646,00004,0,0.1889180,1.0589180,0.1889180
8,00004,4,1,0,0.1889180,2,0.8828973,16004,1,0.1889180,1.0589180,1.0589180
9,00005,5,1,0,-0.3512651,1,2.4524646,00005,0,-0.3512651,0.5187349,-0.3512651
